In [1]:
import pandas as pd
import numpy as np

This script cleans and merges the 911 calls in Baltimore with our base crime data set.

In [125]:
df_911 = pd.read_csv('./data/911_calls_for_service.csv', index_col = 0)
df_911_2017 = pd.read_csv('./data/911_Calls_For_Service_2017.csv')

/Users/tuliprichard/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [142]:
bpd_df = pd.read_csv('./data/BPD_CRIME_DATA.csv', index_col = None)

The datasets from baltimore.gov only have 2017 and onwards, and seem to have more columns than the one from Kaggle (2015-2017) but do not actually providing more information (identifier and repeatitive geographical segmentation).

In [25]:
df_911.head(5)

,callDateTime,priority,district,description,callNumber,incidentLocation,location
0,2015-07-13 10:41:00,Medium,CD,SEE TEXT,P151941002,0 N CALVERT ST,"(39.2899299,-76.6123462)"
1,2015-07-13 10:47:00,Medium,CD,911/NO VOICE,P151941003,600 E FAYETTE ST,"(39.2906737,-76.6071600)"
2,2015-07-13 10:42:00,Medium,CD,911/NO VOICE,P151941004,200 E BALTIMORE ST,"(39.2898910,-76.6120720)"
3,2015-07-13 10:45:00,Low,CD,PRKG COMPLAINT,P151941005,800 PARK AV,"(39.2985163,-76.6184754)"
4,2015-07-13 10:46:00,Medium,SW,AUTO THEFT,P151941006,3500 CLIFTON AV,"(39.3112130,-76.6763150)"


In [126]:
df_911 = df_911[df_911.description != "911/NO  VOICE"]

In [127]:
#df_911_2017.head(5)

In [143]:
bpd_df.head(5)

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,Total Incidents
0,09/02/2017,23:30:00,3JK,4200 AUDREY AVE,ROBBERY - RESIDENCE,I,KNIFE,913.0,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",ROW/TOWNHO,1
1,09/02/2017,23:00:00,7A,800 NEWINGTON AVE,AUTO THEFT,O,NaN,133.0,CENTRAL,Reservoir Hill,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",STREET,1
2,09/02/2017,22:53:00,9S,600 RADNOR AV,SHOOTING,Outside,FIREARM,524.0,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",Street,1
3,09/02/2017,22:50:00,4C,1800 RAMSAY ST,AGG. ASSAULT,I,OTHER,934.0,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",ROW/TOWNHO,1
4,09/02/2017,22:31:00,4E,100 LIGHT ST,COMMON ASSAULT,O,HANDS,113.0,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",STREET,1


We do a quick check whether district and PoliceDistrict are the same. The answer is NO.

In [129]:
len(df_911_2017["district"].unique())

13

In [130]:
len(df_911_2017["PoliceDistrict"].unique())

10

But the PoliceDistrict from the raw datasets seem to mathc the DISTRICT in the crime data set and could be useful for the inner join.    
So let's try to merge based on time, location (street address) first and Longitude and Latitude.	

In [131]:
bpd_df.CrimeDate = pd.to_datetime(bpd_df.CrimeDate.dt.date, format ='%Y-%m-%d')
df_911.callDateTime = pd.to_datetime(df_911.callDateTime)

We crop the crime dataset and only keep crime records decomentaed from 2015-01-01 to 2017-08-22, which leaves us with 130k rows.

In [132]:
bpd_df = bpd_df[(bpd_df.CrimeDate >= "2015-01-01") & (bpd_df.CrimeDate <= "2017-08-22")]
bpd_df.shape

(130051, 15)

In [133]:
df_911.shape

(2342439, 7)

In [134]:
df_911["CALLDAY"] = df_911.callDateTime.dt.date
df_911["CALLDAY_BEFORE"] = df_911.callDateTime.dt.date - datetime.timedelta(days=1)

In [135]:
df_911["CALLDAY"]

0          2015-07-13
3          2015-07-13
4          2015-07-13
5          2015-07-13
6          2015-07-13
              ...    
2799909    2017-08-21
2799910    2017-08-21
2799911    2017-08-21
2799912    2017-08-21
2799913    2017-08-21
Name: CALLDAY, Length: 2342439, dtype: object

In [137]:
pd.options.mode.chained_assignment = None

In [139]:
crime_with_911 = pd.DataFrame()

#try first days
for date in bpd_df.CrimeDate.dt.date.unique()[:1]:
    
    group_crime = bpd_df[bpd_df.CrimeDate.dt.date == date]
    group_call = df_911[(df_911.CALLDAY == date) | (df_911.CALLDAY_BEFORE == date)]
    print(date)
    print(group_crime.shape[0])
    print(group_call.shape[0])
    for j in range(group_crime.shape[0]):
        
        print("next crime")
        for k in range(group_call.shape[0]):
            if group_crime.iloc[j]["Location"] == group_call.iloc[k]["incidentLocation"]:
                new_row = group_crime.iloc[j]
                new_row["callDateTime"] = group_call.iloc[k]["callDateTime"]
                new_row["Priority"] = group_call.iloc[k]["priority"]
                new_row["CallDescription"] = group_call.iloc[k]["description"]

                print("Found One Match")
                crime_with_911.append(new_row)
                
        
        

2017-01-19
156
4932
next crime
next crime
next crime
Found One Match
Found One Match
Found One Match
Found One Match
next crime
next crime
next crime
Found One Match
next crime
next crime
next crime
next crime
next crime
Found One Match
next crime
next crime
next crime
Found One Match
Found One Match
next crime
Found One Match
next crime
next crime
next crime
next crime
Found One Match
Found One Match
Found One Match
Found One Match
Found One Match
Found One Match
next crime
Found One Match
Found One Match
Found One Match
Found One Match
Found One Match
Found One Match
next crime
Found One Match
next crime
next crime
next crime
Found One Match
Found One Match
next crime
Found One Match
Found One Match
Found One Match
Found One Match
next crime
next crime
Found One Match
Found One Match
Found One Match
next crime
Found One Match
Found One Match
Found One Match
next crime
next crime
Found One Match
Found One Match
Found One Match
Found One Match
Found One Match
next crime
next crime
next

In [140]:
crime_with_911

""


In [40]:
#Tried with pandasql
#COULD WORK BUT
#TOOOO SLOW

# import pandasql as ps

# sqlcode = '''
# select *
# FROM bpd_df
# INNOR JOIN df_911
# WHERE (df_911.callDateTime <= bpd_df.CrimeDate + 1 OR df_911.callDateTime >= bpd_df.CrimeDate - 1)
#     AND .... 
# '''

# newdf = ps.sqldf(sqlcode, locals())
